In [ ]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Weighted overlay
Here's a recipe for assigning a quantity from one set of polygons to another, based on the areas of overlap.

In [ ]:
sa2 = gpd.read_file("data/sa2-wellington.gpkg")
school_zones = gpd.read_file("data/school-zones.gpkg")
ax = sa2.plot(alpha = 0.5, ec = "w", lw = 0.5, figsize = (8, 8))
school_zones.plot(ax = ax, fc = "red", alpha = 0.5, ec = "k", lw = 0.5)

To estimate population of the school zones based on population of the SA2 polygons we need their area, so make a copy of the data and add that as a new column with an `assign()` operation.

In [ ]:
sa2_area = sa2.assign(sa2_area = sa2.area)
sa2_area

Next, overlay the school zones and the SA2s, and calculate the areas of the intersections. Here we use the `overlay()` method which is similar to the `sjoin` operation but creates the geospatial unit associated with each of the spatial relations between the two datasets, in this case the intersection of the two sets of polygons. 

Here we use a _lambda function_ which is an anonymous inline function, with a single parameter `x` which stands in for the dataframe to which the `assign()` operation is being applied, i.e., in this case the output from the `overlay()` operation, which we know will have a `geometry` column.

In [ ]:
school_zones_sa2 = school_zones \
    .overlay(sa2_area) \
    .assign(area = lambda x: x.geometry.area)

region = gpd.GeoSeries([sa2_area.union_all().union(school_zones.union_all())])

Now we can estimate populations of the school zones based on the fraction of SA2 area formed by each of these new polygons, make a new dataframe with this information, and merge it back into the school zones data. 

In [ ]:
school_zones_sa2["est_pop"] = \
    school_zones_sa2.CURPop * school_zones_sa2.area / school_zones_sa2.sa2_area
school_zone_pops = school_zones_sa2 \
    .groupby("School_ID", as_index = False) \
    .agg({"est_pop": "sum"})
school_zone_pops = school_zones.merge(school_zone_pops)

In [ ]:
school_zone_pops

And now we can make a map

In [ ]:
ax = region.plot(fc = "#00000000", ec = "k", figsize = (8, 8))
school_zone_pops.plot(
    ax = ax, column = "est_pop", 
    cmap = "Reds", alpha = 0.5, k = 9, scheme = "equalinterval", 
    ec = "k", lw = 0.35, legend = True, legend_kwds = {"loc": "upper left"})
ax.set_axis_off()
plt.show()

## Wrap it in a function
Again, it is instructive to think about how to wrap all this in a function. Here's one possibility:

In [ ]:
def area_interpolation(
    src:gpd.GeoDataFrame,
    dst:gpd.GeoDataFrame,
    vars:list[str],
    id_var:str) -> gpd.GeoDataFrame:
    """Returns estimates of extensive variables based on areas of overlap.

    Args:
        src (gpd.GeoDataFrame): source gdf with original variable estimates.
        dst (gpd.GeoDataFrame): destination gdf of areas where estimates are to
            be made.
        vars (list[str]): variable names for which estimates are required.
        id_var (str): unique id attribute in the destination gdf.

    Returns:
        gpd.GeoDataFrame: new gdf with estimates of requested variables, which
            will be prefixed 'est_'.
    """
    new_vars = {v: f"est_{v}" for v in vars}
    src_area = src.assign(src_area = src.area)
    overlay = dst \
        .overlay(src_area) \
        .assign(area = lambda x: x.geometry.area)
    for v, new_v in new_vars.items():
        overlay[new_v] = overlay[v] * overlay.area / overlay.src_area
    return dst.merge(
        overlay \
            .groupby(id_var, as_index = False) \
            .agg({v: "sum" for v in new_vars.values()}) )

area_interpolation(sa2, school_zones, ["CURPop"], "School_ID")